In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
#自定义层结构
class MyDense(keras.layers.Layer):
    def __init__(self,in_dim,out_dim):
        super(MyDense,self).__init__()
        self.kernel = self.add_variable('w',[in_dim,out_dim])
        self.bias = self.add_variable('b',[out_dim])
    
    def call(self,input,training=None):
        out = input@self.kernel+self.bias
        return out

In [3]:
#自定义网络
class MyModel(keras.Model):

    def __init__(self):
        super(MyModel, self).__init__()

        self.fc1 = MyDense(784, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)

    def call(self, inputs, training=None):

        x = self.fc1(inputs)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.relu(x)
        x = self.fc3(x)
        x = tf.nn.relu(x)
        x = self.fc4(x)
        x = tf.nn.relu(x)
        x = self.fc5(x)
        x = tf.nn.softmax(x)

        return x

In [4]:
#载入数据
(x_train,y_train),(x_val,y_val)=keras.datasets.fashion_mnist.load_data()
print('训练集维度：',x_train.shape,y_train.shape)
print('数据范围:',x_train.min(),x_train.max())
print('类别：',set(y_train))
print('类别数：',len(set(y_train)))
print('验证集维度：',x_val.shape,y_val.shape)

训练集维度： (60000, 28, 28) (60000,)
数据范围: 0 255
类别： {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
类别数： 10
验证集维度： (10000, 28, 28) (10000,)


In [5]:
#预处理数据
def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)/255
    x = tf.reshape(x,[-1,784])
    y = tf.cast(y,dtype=tf.int32)
    y = tf.one_hot(y,depth=10)
    return x,y

In [6]:
#准备每次喂入网络的数据batch
#每个batch200组数据，用preprocess处理这些数据，随机打乱（打乱时内存分配空间为10000）
db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(200).map(preprocess).shuffle(10000)
db_val = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(200).map(preprocess).shuffle(10000)

db_iter = iter(db_train)
sample = next(db_iter)
print('每次喂入的样本量:',sample[0].shape)
print('每次喂入的标签量:',sample[1].shape)

每次喂入的样本量: (200, 784)
每次喂入的标签量: (200, 10)


In [7]:
#实例化网络
model = MyModel()
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),loss=tf.losses.categorical_crossentropy,metrics=['accuracy'])
model.fit(db_train,epochs=10,validation_data=db_val,validation_steps=2)

Epoch 1/10
300/300 [==============================] - 4s 12ms/step - loss: 0.6225 - accuracy: 0.6722 - val_loss: 0.5790 - val_accuracy: 0.8100
Epoch 2/10
300/300 [==============================] - 2s 8ms/step - loss: 0.4065 - accuracy: 0.8505 - val_loss: 0.5428 - val_accuracy: 0.8100
Epoch 3/10
300/300 [==============================] - 2s 8ms/step - loss: 0.3712 - accuracy: 0.8660 - val_loss: 0.5314 - val_accuracy: 0.8075
Epoch 4/10
300/300 [==============================] - 2s 8ms/step - loss: 0.3471 - accuracy: 0.8770 - val_loss: 0.5418 - val_accuracy: 0.8275
Epoch 5/10
300/300 [==============================] - 3s 8ms/step - loss: 0.3366 - accuracy: 0.8795 - val_loss: 0.5254 - val_accuracy: 0.8400
Epoch 6/10
300/300 [==============================] - 3s 9ms/step - loss: 0.3270 - accuracy: 0.8850 - val_loss: 0.5368 - val_accuracy: 0.8400
Epoch 7/10
300/300 [==============================] - 3s 9ms/step - loss: 0.3197 - accuracy: 0.8836 - val_loss: 0.5108 - val_accuracy: 0.8375
Epoch

In [8]:
model.compile(optimizer=tf.optimizers.SGD(learning_rate=0.01),loss=tf.losses.categorical_crossentropy,metrics=['accuracy'])
model.fit(db_train,epochs=10,validation_data=db_val,validation_steps=2)

Epoch 1/10
300/300 [==============================] - 3s 8ms/step - loss: 0.2728 - accuracy: 0.8997 - val_loss: 0.4806 - val_accuracy: 0.8475
Epoch 2/10
300/300 [==============================] - 2s 6ms/step - loss: 0.2613 - accuracy: 0.9036 - val_loss: 0.4793 - val_accuracy: 0.8500
Epoch 3/10
300/300 [==============================] - 2s 7ms/step - loss: 0.2563 - accuracy: 0.9048 - val_loss: 0.4796 - val_accuracy: 0.8500
Epoch 4/10
300/300 [==============================] - 2s 7ms/step - loss: 0.2531 - accuracy: 0.9056 - val_loss: 0.4793 - val_accuracy: 0.8500
Epoch 5/10
300/300 [==============================] - 2s 7ms/step - loss: 0.2507 - accuracy: 0.9067 - val_loss: 0.4786 - val_accuracy: 0.8550
Epoch 6/10
300/300 [==============================] - 2s 7ms/step - loss: 0.2488 - accuracy: 0.9072 - val_loss: 0.4777 - val_accuracy: 0.8550
Epoch 7/10
300/300 [==============================] - 2s 7ms/step - loss: 0.2473 - accuracy: 0.9080 - val_loss: 0.4767 - val_accuracy: 0.8575
Epoch 

In [9]:
model.evaluate(db_val)

50/50 [==============================] - 0s 5ms/step - loss: 0.3505 - accuracy: 0.8814


[0.35045804679393766, 0.8814]